#### UTILITARY SCRIPT TO CONVERT IMAGES IN AGGREGATED DATA

In [38]:
import pandas as pd
import numpy as np
import json
import itertools


In [28]:
DIM_AGREGATION = 25 # X 25 meters

Import JSON File of satellites images :

In [11]:
with open(f"Results_Images_Stock/Images_LANDSAT_LFI4.json", 'r') as openfile:
 
    data = json.load(openfile)

Verifications :

In [90]:
data['51']['LFI']

'LFI4'

In [26]:
np.shape(data['51']['IMAGES_SAT']['NDVI'])

(200, 200)

Import list of parcelles :

In [92]:
df = pd.read_excel('DataFrames/data_parcelles_with_gps.xlsx')

In [93]:
df_result = df[['PARCELLE','LFI']]

LFI Agregation (if error : correspondance false)

In [148]:
LFI = data['51']['LFI']
df_result = df_result.loc[df_result['LFI']==LFI,:].reset_index(drop=True)

In [103]:
df_result.head()

,PARCELLE,LFI
0,51,LFI4
1,384,LFI4
2,1239,LFI4
3,1419,LFI4
4,1431,LFI4


Constants definitions :

In [139]:
nb_parc = len(df_result)
problematic_parcelles = []
list_parcelle = df_result['PARCELLE'].tolist()
width = np.shape(data['51']['IMAGES_SAT']['NDVI'])[0]
height = np.shape(data['51']['IMAGES_SAT']['NDVI'])[1]
range_w = np.arange(int((width - DIM_AGREGATION)/2), int((width + DIM_AGREGATION)/2))
range_h = np.arange(int((height - DIM_AGREGATION)/2), int((height + DIM_AGREGATION)/2))
ndvi_arr = np.empty(nb_parc)
evi_arr = np.empty(nb_parc)
ndmi_arr = np.empty(nb_parc)
ndwi_arr = np.empty(nb_parc)
dswi_arr = np.empty(nb_parc)
MAPPING = {
    'NDVI' : ndvi_arr,
    'EVI' : evi_arr,
    'NDMI' : ndmi_arr,
    'NDWI' : ndwi_arr,
    'DSWI' : dswi_arr
}

for ind, parc in enumerate(list_parcelle):
    try:
        for func_name, arr in MAPPING.items():
            pixel_values = data[str(parc)]['IMAGES_SAT'][func_name]
            arr[ind] = round(np.mean([pixel_values[tupl[0]][tupl[1]] for tupl in list(itertools.product(range_w,range_h))]),4)
    except:
        problematic_parcelles.append(parc)
        for _ , arr in MAPPING.items():
            arr[ind] = np.nan

df_result['NDVI'] = ndvi_arr
df_result['EVI'] = evi_arr
df_result['NDMI'] = ndmi_arr
df_result['NDWI'] = ndwi_arr
df_result['DSWI'] = dswi_arr

In [151]:
df_result.to_excel(f'./DATA_aggregated/Data_from_satellites_images_{LFI}.xlsx')